In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')


## For save/read files

In [2]:
def read_file(file_name):
    df = pd.read_csv(file_name, sep=';')
    pd.set_option('display.max_columns', 125)
    return df

In [3]:
def save_file(all_avto_links, file_name):
    df = pd.DataFrame({'pages_href': all_avto_links}, index=range(1, len(all_avto_links)+1))
    df = df.rename_axis('id_car')
    df.head(100)

    df.to_csv(file_name, sep=';')

In [61]:
def save_df(df, file_name):
    df.to_csv(file_name, sep=';')

## For parse links of announcement of cars

In [4]:
def parse_links(url, all_avto_links):
    response = requests.get(url)
    
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    avtos = soup.find_all('div', {'class':'listing-item'})
    
    for avto in avtos:
        class_pages_href = avto.find('a')['href']    
        all_avto_links.append(class_pages_href)
        
    return all_avto_links
    

In [7]:
url = 'https://cars.av.by/search/page/'

file_name = 'data/auto13.csv'

all_avto_links = []
    
for page in range(1, 50):
    all_avto_links = parse_links(url+str(page), all_avto_links)
    

save_file(all_avto_links, file_name)
read_file(file_name)

,id_car,pages_href
0,1,https://cars.av.by/volkswagen/polo-sedan/17136596
1,2,https://cars.av.by/volkswagen/golf/17092401
2,3,https://cars.av.by/buick/encore/16826264
3,4,https://cars.av.by/skoda/fabia/16737637
4,5,https://cars.av.by/mitsubishi/pajero/16790831
...,...,...
1220,1221,https://cars.av.by/audi/a6-allroad/16722800
1221,1222,https://cars.av.by/fiat/500/16369546
1222,1223,https://cars.av.by/nissan/primera/16655888
1223,1224,https://cars.av.by/bmw/3-seriya/16423759


## For parse data 

In [66]:
features_avto = ['ABS (антиблокировочная система)',
     'AUX/iPod',
     'Bluetooth',
     'CD/MP3 проигрыватель',
     'ESP (система поддержания динамической стабильности)',
     'USB',
     'Автозапуск двигателя',
     'Антипробуксовочная система',
     'Датчик дождя',
     'Дополнительные опции:',
     'Иммобилайзер',
     'Камера заднего вида',
     'Климат-контроль',
     'Кондиционер',
     'Контроль мертвых зон на зеркалах',
     'Круиз-контроль',
     'Ксеноновые фары',
     'Легкосплавные диски',
     'Люк',
     'Материал салона - ',
     'Мультимедийный экран',
     'Обогрев зеркал',
     'Обогрев лобового стекла',
     'Обогрев руля',
     'Обогрев сидений',
     'Панорамная крыша',
     'Парктроники',
     'Подушки безопасности боковые',
     'Подушки безопасности задние',
     'Подушки безопасности передние',
     'Противотуманные фары',
     'Рейлинги на крыше',
     'Светодиодные фары',
     'Сигнализация',
     'Системы помощи',
     'Управление мультимедиа с руля',
     'Фаркоп',
     'Цвет салона - темный',
     'Штатная навигация',
     'Электрорегулировка сидений',
     'Электростеклоподъемники задние',
     'Электростеклоподъемники передние',
     'Материал салона - натуральная кожа']

main_features_avto = ['Год',
                      'Состояние',
                      'Пробег',
                      'Тип топлива',
                      'Объем',
                      'Цвет',
                      'Тип кузова',
                      'Трансмиссия',
                      'Привод']


In [72]:
df = pd.DataFrame(columns=(['Название', 'Цена', 'Регион'] + main_features_avto + features_avto))

In [73]:
def parse(url):
    response = requests.get(url)
    response.encoding = 'utf8'
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    avtos = soup.find_all('div', {'class':'card'})

    avto = []
    for elem in avtos:
        name = re.sub(r'<h1 class=\"card-title\">|</h1>|\n*', 
                      r'', str(elem.find('h1', {'class':'card-title'})).strip())
        avto.append(name)

        price = re.sub(r'[<>a-zA-Z\/\"\'\=\-\+\_\@\#\/\.\n\ ]*', 
                       r'',str(elem.find('span', {'class':'card-price-main-primary'})).strip())
        avto.append(price)
        
        town = re.sub(r'<h4 class=\"card-contacts-city\">|</h4>|\n*', r'', 
                      str(elem.find('h4', {'class':'card-contacts-city'})).strip())
        avto.append(town)

        main_info = str(elem.find('div', {'class':'card-info'})).strip()
        options = re.sub(r'[<>a-zA-Z\/\"\'\=\-\+\_\@\#\/\.\,]', r'', main_info)
        fin_str = re.sub(r'[\n]', r' ', options).strip()
        options = fin_str.split('     ')        
        for element in options:
            avto.append(element)
                
        add_f = re.sub(r'<div class=\"card-options js-card-options\">|<h4>|<\/h4>|<h5>|<\/h5>|<div>|<\/div>|<ul>|<\/ul>|<\/li>|Дополнительные опции:|\n*', r'', 
                       str(elem.find('div', {'class':'card-options js-card-options'})).strip())

        add_f = add_f.split('<li>')        
        for elem in add_f:
            avto.append(elem)
            
        return avto

In [74]:
def df_insert(dd):
    
    element_name = dd.pop(0)
    element_price = dd.pop(0)
    element_town = dd.pop(0)
    temp = [element_name, element_price, element_town]
    
    for element in main_features_avto + features_avto:
        flag = True
        for elem in dd:
            if elem.startswith(element):
                temp.append(elem)                
                flag = False
                break
                
        if flag:
            temp.append(None)
    
    df.loc[len(df)] = temp


## Loading data

In [75]:
multi_class = 'card'

# 14
for elem in range(8, 14):
    from_filename = 'data/auto'+str(elem)+'.csv'
    df_link = read_file(from_filename)
    df_link['pages_href'] = df_link['pages_href'].apply(str)
    
    for url in df_link['pages_href']:    
        dd = parse(url)
        try:
            df_insert(dd)
        except :
            print(url)
        
    to_filename = 'data/av'+str(elem)+'.csv'
    save_df(df, to_filename)

In [65]:
df.head(100)

,Название,Цена,Регион,Год,Состояние,Пробег,Тип топлива,ОбЪем,Цвет,Тип кузова,Трансмиссия,Привод,ABS (антиблокировочная система),AUX/iPod,Bluetooth,CD/MP3 проигрыватель,ESP (система поддержания динамической стабильности),USB,Автозапуск двигателя,Антипробуксовочная система,Датчик дождя,Дополнительные опции:,Иммобилайзер,Камера заднего вида,Климат-контроль,Кондиционер,Контроль мертвых зон на зеркалах,Круиз-контроль,Ксеноновые фары,Легкосплавные диски,Люк,Материал салона -,Мультимедийный экран,Обогрев зеркал,Обогрев лобового стекла,Обогрев руля,Обогрев сидений,Панорамная крыша,Парктроники,Подушки безопасности боковые,Подушки безопасности задние,Подушки безопасности передние,Противотуманные фары,Рейлинги на крыше,Светодиодные фары,Сигнализация,Системы помощи,Управление мультимедиа с руля,Фаркоп,Цвет салона - темный,Штатная навигация,Электрорегулировка сидений,Электростеклоподъемники задние,Электростеклоподъемники передние,Материал салона - натуральная кожа
0,"Volkswagen Passat B6, 2006 ...",17324р,Минск,Год выпуска 2006,Состояние с пробегом,Пробег 303656 км,Тип топлива дизель,None,Цвет синий,Тип кузова универсал,Трансмиссия Автомат,Привод Передний,ABS (антиблокировочная система),None,None,None,None,None,None,None,Датчик дождяОптика и свет,None,Иммобилайзер,None,Климат-контрольБезопасность,None,None,Круиз-контрольКлимат и обогрев,None,Легкосплавные дискиСистемы помощи,None,Материал салона - ткань,None,None,None,None,Обогрев сидений,None,None,Подушки безопасности боковые,None,Подушки безопасности передние,Противотуманные фарыМультимедиа,Рейлинги на крыше,None,None,None,Управление мультимедиа с руля,None,Цвет салона - темныйЭкстерьер,Штатная навигацияКомфорт,Электрорегулировка сидений,Электростеклоподъемники задние,Электростеклоподъемники передние,None
1,"Skoda Octavia II (рестайлинг),...",20294р,Минск,Год выпуска 2011,Состояние с пробегом,Пробег 195000 км,Тип топлива бензин,None,Цвет серый,Тип кузова универсал,Трансмиссия Механика,Привод Передний,ABS (антиблокировочная система),AUX/iPodКомфорт,None,CD/MP3 проигрыватель,ESP (система поддержания динамической стабильн...,None,None,None,None,None,Иммобилайзер,None,Климат-контрольБезопасность,Кондиционер,None,None,None,Легкосплавные дискиСистемы помощи,None,Материал салона - ткань,None,Обогрев зеркал,None,None,None,None,ПарктроникиОптика и свет,None,Подушки безопасности задние,Подушки безопасности передние,Противотуманные фарыМультимедиа,Рейлинги на крыше,None,None,None,None,Фаркоп,Цвет салона - темныйЭкстерьер,None,None,Электростеклоподъемники задниеКлимат и обогрев,Электростеклоподъемники передние,None
2,"Volkswagen Passat B6, 2006 ...",17324р,Минск,Год выпуска 2006,Состояние с пробегом,Пробег 303656 км,Тип топлива дизель,None,Цвет синий,Тип кузова универсал,Трансмиссия Автомат,Привод Передний,ABS (антиблокировочная система),None,None,None,None,None,None,None,Датчик дождяОптика и свет,None,Иммобилайзер,None,Климат-контрольБезопасность,None,None,Круиз-контрольКлимат и обогрев,None,Легкосплавные дискиСистемы помощи,None,Материал салона - ткань,None,None,None,None,Обогрев сидений,None,None,Подушки безопасности боковые,None,Подушки безопасности передние,Противотуманные фарыМультимедиа,Рейлинги на крыше,None,None,None,Управление мультимедиа с руля,None,Цвет салона - темныйЭкстерьер,Штатная навигацияКомфорт,Электрорегулировка сидений,Электростеклоподъемники задние,Электростеклоподъемники передние,None
3,"BMW X3 F25, 2012",37126р,Минск,Год выпуска 2012,Состояние с пробегом,Пробег 192800 км,Тип топлива бензин,None,Цвет серебристый,Тип кузова внедорожник 5 дв,Трансмиссия Автомат,Привод Постоянный полный,ABS (антиблокировочная система),AUX/iPodКомфорт,None,CD/MP3 проигрыватель,ESP (система поддержания динамической стабильн...,None,Автозапуск двигателяКлимат и обогрев,None,Датчик дождяОптика и свет,None,Иммобилайзер,None,None,КондиционерБезопасность,None,None,None,Легкосплавные дискиСистемы помощи,None,Материал салона - искусственная кожа,Мультимедийный экран,Обогрев зеркал,None,None,Обогрев сиден